In [2]:
import torch

In [21]:
t = torch.randn(128, 5)
t

tensor([[-1.9111e-01,  8.2067e-01,  1.2404e-01, -1.4581e-01, -4.1619e-01],
        [ 3.8760e-01, -3.1325e-01, -3.1595e-01,  2.2876e-01, -1.3623e+00],
        [-8.1782e-01, -3.4508e-01,  6.0888e-01,  3.2214e-01, -7.7165e-01],
        [-1.2035e-01, -6.1502e-01,  1.3577e+00,  6.1687e-02, -8.4045e-01],
        [ 8.1171e-01, -1.4848e+00, -3.4276e-01, -1.1641e+00,  1.7970e-01],
        [-1.3730e+00, -1.1077e+00, -2.1760e+00, -2.3773e-01, -4.2470e-01],
        [-5.1302e-01, -3.8826e-01,  5.0159e-01,  1.5499e+00,  1.1484e-01],
        [ 6.2127e-01, -3.9661e-01, -1.5698e+00,  8.5453e-01, -1.3641e+00],
        [ 4.4265e-02,  8.7642e-01,  1.6834e+00, -1.3516e+00, -1.1120e+00],
        [-5.9527e-01, -2.4350e-01, -2.0747e-01,  1.2280e+00,  5.0158e-01],
        [ 5.6715e-02, -1.3630e+00,  2.6531e-02, -1.5513e+00, -5.2003e-01],
        [ 2.0965e+00,  4.1094e-01, -6.9360e-01, -9.1369e-01, -1.1872e-01],
        [ 1.8276e+00,  3.1915e-01,  1.4969e+00,  8.0538e-02,  1.6665e-01],
        [-8.9657e-01,  7.

In [24]:
s = torch.nn.functional.log_softmax(t, dim=1)
s

tensor([[-1.9410, -0.9292, -1.6259, -1.8957, -2.1661],
        [-1.1046, -1.8054, -1.8081, -1.2634, -2.8544],
        [-2.3927, -1.9200, -0.9660, -1.2528, -2.3465],
        [-2.0387, -2.5334, -0.5606, -1.8567, -2.7588],
        [-0.7352, -3.0317, -1.8897, -2.7111, -1.3673],
        [-2.1336, -1.8683, -2.9366, -0.9983, -1.1853],
        [-2.6833, -2.5586, -1.6687, -0.6204, -2.0555],
        [-1.0554, -2.0733, -3.2465, -0.8222, -3.0408],
        [-2.1985, -1.3663, -0.5593, -3.5943, -3.3548],
        [-2.5712, -2.2195, -2.1835, -0.7480, -1.4744],
        [-1.0899, -2.5097, -1.1201, -2.6980, -1.6667],
        [-0.3401, -2.0257, -3.1303, -3.3504, -2.5554],
        [-0.8346, -2.3431, -1.1653, -2.5817, -2.4956],
        [-2.7290, -1.7596, -0.7492, -2.0865, -1.7973],
        [-3.5645, -2.6035, -0.3734, -2.2910, -2.2246],
        [-0.7375, -1.6140, -1.8953, -2.3716, -2.5380],
        [-0.4685, -2.6031, -2.2899, -2.5182, -2.1358],
        [-1.5346, -1.2392, -3.0463, -2.2309, -1.0792],
        [-

In [27]:
torch.sum(torch.exp(s), dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 